In [5]:
import nibabel as nib
import numpy as np
import torch as th
from monai.transforms import Compose, LoadImage, CropForeground, EnsureChannelFirst, ResizeWithPadOrCrop, ScaleIntensityRange
def get_tensor(file_path, norm):
    # Step 1: Load the image and metadata using nibabel
    transforms = [
        LoadImage(image_only=True),
        EnsureChannelFirst()
        ]
    if norm:
        transforms.append(
        ScaleIntensityRange(a_min=-200, a_max=200, b_min=-1, b_max=1, clip=False)
        )

    apply_transforms = Compose(transforms)
    np_tensor = apply_transforms(file_path)[0].numpy()
    return np_tensor

def get_segmentation(file_path):
    transforms = Compose([
        LoadImage(image_only=True),
        EnsureChannelFirst(),
        CropForeground(select_fn=lambda x: x > 0, margin=0),
        ResizeWithPadOrCrop(spatial_size=(128,128,128))
    ])
    segmentation = transforms(file_path)[0].numpy()
    return segmentation

import nibabel as nib

def get_affine_and_header(file_path):
  """
  Extracts the affine transformation matrix and header information from a NIfTI file.

  Args:
    filename (str): The path to the NIfTI file.

  Returns:
    tuple: A tuple containing the affine matrix and header information.
  """

  img = nib.load(file_path)
  affine = img.affine
  header = img.header

  return affine, header

In [7]:
healthy_ct_scan_full_res = get_tensor(
                    file_path="/home/andreferreira/Desktop/AI_work/Aachen/TotalSeg/head-neck-ct_fake/anderson_0a908279226c5229e7fe85b8894b62d5_tumour_Contrast.nii.gz",
                    norm=True
                    )
region_to_place_tumour_mask = get_tensor(
    file_path="/home/andreferreira/Desktop/AI_work/Aachen/TotalSeg/totalseg_output/head-neck-ct_fake/post_processed/anderson_0a908279226c5229e7fe85b8894b62d5_tumour_Contrast.nii.gz",
    norm=False
    )
segmentation = get_segmentation(
    file_path="/home/andreferreira/Desktop/AI_work/Aachen/TotalSeg/anderson_0a696b5b3901ce97c24828e4962c7267.nii.gz"
    )

affine, header = get_affine_and_header(file_path="/home/andreferreira/Desktop/AI_work/Aachen/TotalSeg/head-neck-ct_fake/anderson_0a908279226c5229e7fe85b8894b62d5_tumour_Contrast.nii.gz")
print("well")

well


/home/andreferreira/anaconda3/envs/totalseg/lib/python3.11/site-packages/monai/utils/deprecate_utils.py:321: FutureWarning: monai.transforms.croppad.array CropForeground.__init__:allow_smaller: Current default value of argument `allow_smaller=True` has been deprecated since version 1.2. It will be changed to `allow_smaller=False` in version 1.5.
  warn_deprecated(argname, msg, warning_category)


In [8]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
import nibabel as nib
import numpy as np

# Select a random voxel to be the center of the tumour
voxel_indices = np.argwhere(region_to_place_tumour_mask == 2)
random_voxel_indices = voxel_indices[np.random.choice(len(voxel_indices), size=1, replace=False)]
print(f"New center: {random_voxel_indices}")
random_x, random_y, random_z = random_voxel_indices[0][0], random_voxel_indices[0][1], random_voxel_indices[0][2]

# Crop the full resolution scan and mask
ct_scan_crop = healthy_ct_scan_full_res[ random_x-64:random_x+64,
                        random_y-64:random_y+64,
                        random_z-64:random_z+64]

region_to_place_tumour_mask_crop = region_to_place_tumour_mask[ random_x-64:random_x+64,
                        random_y-64:random_y+64,
                        random_z-64:random_z+64]

# Ensure the segmentation remains within the anatomical boundaries defined by the region_to_place_tumour_mask_crop
corrected_seg = segmentation.copy()
corrected_seg[region_to_place_tumour_mask_crop == 1] = 0


img = nib.Nifti1Image(ct_scan_crop, affine=np.eye(4))
nib.save(img, '/home/andreferreira/Desktop/AI_work/Aachen/TotalSeg/trash/ct_scan_crop.nii.gz')
img = nib.Nifti1Image(region_to_place_tumour_mask_crop, affine=affine, header=header)
nib.save(img, '/home/andreferreira/Desktop/AI_work/Aachen/TotalSeg/trash/region_to_place_tumour_mask_crop.nii.gz')
img = nib.Nifti1Image(segmentation,affine=affine, header=header)
nib.save(img, '/home/andreferreira/Desktop/AI_work/Aachen/TotalSeg/trash/segmentation.nii.gz')
img = nib.Nifti1Image(corrected_seg, affine=affine, header=header)
nib.save(img, '/home/andreferreira/Desktop/AI_work/Aachen/TotalSeg/trash/corrected_seg.nii.gz')



New center: [[ 86 160 147]]
